In [ ]:
from parse_deck import parse_deck

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import time
import re
from tqdm import tqdm

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--start-maximized")

In [ ]:
# parse CL event links from official website
url = "https://players.pokemon-card.com/event/result/list"
driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(10) # seconds
driver.get(url)
events = driver.find_elements(By.CLASS_NAME, "eventListItem")

# collect events for city league (シティリーグ)
cl_events = []
cl_event_links = []
for event in tqdm(events):
    title = event.find_element(By.CLASS_NAME, "title")
    if "シティリーグ" in title.text:
        no_people_str = event.find_element(By.CLASS_NAME, "capacity").text
        cl_events.append(
            {
                "link": event.get_attribute("href"),
                "no_people": re.findall(r'\d+', no_people_str),
            }
        )

driver.close()

In [ ]:
# update deck info from CL events
# decks = {
#     deck category 1: [
#         {
#             deck_code: "",
#             pokemons: {},
#             tools: {},
#             supporters: {},
#             stages: {},
#             energies: {},
#             rank: 1,
#             no_people: 32,
#             date: datetime
#         }, ...
#     ],
#     deck category 2:...
# }

categories = ["ルギアVSTAR", "ミュウVMAX", "ジュラルドンVMAX", "others"]
decks = {}
for c in categories:
    decks[c] = []

#
for cl_event in tqdm(cl_events):
    cl_link = cl_event["link"]
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(10) # seconds
    driver.get(cl_link)

    date_str = driver.find_element(By.CLASS_NAME, "date-day").text
    rows = driver.find_elements(By.CLASS_NAME, "c-rankTable-row")
    for row in rows:
        deck_link = row.find_element(By.CLASS_NAME, "deck").find_element(By.TAG_NAME, "a").get_property("href")
#         print(deck_link)
        pokemon_dict, tool_dict, supporter_dict, stage_dict, energy_dict = parse_deck(deck_link = deck_link)

        # identify this deck belongs to which category (TODO: need more rules)
        category = "others"
        for c in categories[:-1]:
            if c in pokemon_dict.keys():
                category = c

        rank = row.find_element(By.TAG_NAME, "td").get_attribute("class").split("-")[-1]
        decks[category].append(
            {
                "deck_link": deck_link,
                "pokemons": pokemon_dict,
                "tools": tool_dict,
                "supporters": supporter_dict,
                "stages": stage_dict,
                "energies": energy_dict,
                "rank": rank,
                "no_people": cl_event["no_people"][0],
                "date": date_str
            }
        )

In [ ]:
decks["others"] 

In [ ]:
deck_name = []
for d in decks["others"]:
    lead = list(d["pokemons"].keys())[0]
    if lead not in deck_name:
        deck_name.append(lead)
        
print(deck_name)


In [ ]:
for d in decks["others"]:
    lead = list(d["pokemons"].keys())[0]
    if lead == "アルセウスV":
        print(d)
        print(d["deck_link"])
        print("\n---\n")